In [ ]:
import numpy as np 
from PIL import Image 
import matplotlib.pyplot as plt 
import os 
import torch as T 
import torch.nn as nn 
import torch.nn.functional as F 
import torchvision.transforms as trans
from torch.utils.data import Dataset, DataLoader

# Clobal Variable

In [ ]:
mode = 'sngan' # 'wgan', 'wgan-gp', 'sngan'

path_image = "/home/wwang/datasets/ship/data/images/"
crop_size = 320
image_size = 64
channel_g = 64
channel_d = 64
row = 2
col = 4
latent_dim = 100
device = T.device("cuda:0")
fixed_noise = T.randn([row*col, latent_dim, 1, 1]).to(device)
batch_size = 3
epochs = 999
learning_rate_d = 2e-4
learning_rate_g = 2e-4
step_g = 1
λ_gp = 10
path_work = '/home/wwang/wwgeneration/work/' + mode + '/'
num_workers = 4

# Data

In [ ]:
class Dataset_wgan(Dataset):
    #
    def __init__(self, path=path_image):
        self.path = path
        self.dirs = os.listdir(path)
        self.transform = trans.Compose([trans.RandomCrop(crop_size),
                                        trans.Resize(image_size, Image.BICUBIC),
                                        trans.RandomHorizontalFlip(),
                                        trans.RandomVerticalFlip(),
                                        trans.RandomRotation([90,90]),
                                        trans.ToTensor(),
                                        trans.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                        ])
    #
    def __getitem__(self, idx):
        image = Image.open(self.path + self.dirs[idx]).convert('RGB')
        image = self.transform(image)
        return image
    #
    def __len__(self):
        return len(self.dirs)

In [ ]:
dataloader = DataLoader(Dataset_wgan(), batch_size=1)
n = 0
for image in dataloader:
    if n == 2:
        break
    image = image.numpy().squeeze().transpose([1,2,0])
    plt.subplot(1,2,1), plt.imshow(image/2+0.5)
    plt.show()
    n += 1

# Network Structure

## Generator

In [ ]:
class CNA(nn.Module):
    #
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, conv, norm, act, mode=False):
        super().__init__()
        if conv == 'conv':
            if mode == 'sngan':
                self.layers = nn.ModuleList([T.nn.utils.parametrizations.spectral_norm(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False))])
            else:
                self.layers = nn.ModuleList([nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)])
        if conv == 'tconv':
            self.layers = nn.ModuleList([nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)])
        if norm == 'bn':
            self.layers.append(nn.BatchNorm2d(out_channels))
        if act == 'relu':
            self.layers.append(nn.ReLU())
        if act == 'lrelu':
            self.layers.append(nn.LeakyReLU(0.2))
    #
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
class Generator(nn.Module):
    #
    def __init__(self, latent_dim=latent_dim, channel=channel_g):
        super().__init__()
        self.layers = nn.Sequential(
            CNA(latent_dim, channel*8, 4, 1, 0, 'tconv', 'bn', 'relu'),
            CNA(channel*8, channel*4, 4, 2, 1, 'tconv', 'bn', 'relu'),
            CNA(channel*4, channel*2, 4, 2, 1, 'tconv', 'bn', 'relu'),
            CNA(channel*2, channel, 4, 2, 1, 'tconv', 'bn', 'relu'),
            CNA(channel, 3, 4, 2, 1, 'tconv', False, False),
            nn.Tanh()
        )
    #
    def forward(self, x):
        return self.layers(x)

## Discriminator

In [ ]:
class Discriminator(nn.Module):
    #
    def __init__(self, channel=channel_d, mode=mode):
        super().__init__()
        self.layers = nn.Sequential(
            CNA(3, channel, 4, 2, 1, 'conv', False, 'lrelu', mode=mode),
            CNA(channel, channel*2, 4, 2, 1, 'conv', False, 'lrelu', mode=mode),
            CNA(channel*2, channel*4, 4, 2, 1, 'conv', False, 'lrelu', mode=mode),
            CNA(channel*4, channel*8, 4, 2, 1, 'conv', False, 'lrelu', mode=mode),
            CNA(channel*8, 1, 4, 1, 0, 'conv', False, False, mode=mode)
        )
    #
    def forward(self, x):
        return self.layers(x).reshape([-1,1])

# Show

In [ ]:
@T.no_grad()
def show(net=None, row=2, col=4, fix=True):
    if net is None:
        net = Generator().to(device)
    # net.eval()
    if fix:
        images = net(fixed_noise).cpu().numpy().transpose([0,2,3,1])
    else:
        images = net(T.randn([row*col, latent_dim, 1, 1]).to(device)).cpu().numpy().transpose([0,2,3,1])
    print(np.max(images), np.min(images))
    plt.figure(figsize=(16,8))
    for i in range(row*col):
        image = (images[i,:,:,:] + 1) / 2
        plt.subplot(row, col, i+1), plt.imshow(image)
    plt.show()

In [ ]:
show()

# Train

In [ ]:
def gradient_penalty(y, x):
    """Compute gradient penalty: (L2_norm(dy/dx) - 1)**2."""
    weight = T.ones(y.shape).to(device)
    dydx = T.autograd.grad(y, x, grad_outputs=weight, retain_graph=True, create_graph=True)[0]
    dydx = dydx.reshape([dydx.shape[0], -1])
    dydx_l2norm = T.sqrt(T.sum(dydx**2, axis=1))
    return T.mean((dydx_l2norm-1)**2)

In [ ]:
def trainer_d(net_d, optimizer_d, net_g, image_real):
    #
    src_real = net_d(image_real)
    image_fake = net_g(T.randn([image_real.shape[0], latent_dim, 1, 1]).to(device))
    src_fake = net_d(image_fake.detach())
    #
    loss = T.mean(-src_real) + T.mean(src_fake)
    #
    if mode == 'wgan-gp':
        α = T.rand([image_real.shape[0], 1, 1, 1]).to(device)
        image_real_fix_fake = α*image_real + (1-α)*image_fake
        src_real_fix_fake = net_d(image_real_fix_fake)
        loss += λ_gp*gradient_penalty(src_real_fix_fake, image_real_fix_fake)
    #
    optimizer_d.zero_grad()
    loss.backward()
    optimizer_d.step()
    #
    if mode == 'wgan':
        for p in net_d.parameters():
            p.data.clamp_(-0.01, 0.01)

In [ ]:
def trainer_g(net_g, optimizer_g, net_d, image_real):
    #
    image_fake = net_g(T.randn([image_real.shape[0], latent_dim, 1, 1]).to(device))
    src_fake = net_d(image_fake)
    #
    loss = T.mean(-src_fake)
    #
    optimizer_g.zero_grad()
    loss.backward()
    optimizer_g.step()

In [ ]:
def train(epochs, load_model):
    #
    net_d = Discriminator().to(device)
    net_g = Generator().to(device)
    if mode == 'wgan':
        optimizer_d = T.optim.RMSprop(net_d.parameters(), learning_rate_d)
        optimizer_g = T.optim.RMSprop(net_g.parameters(), learning_rate_g)
    else:
        optimizer_d = T.optim.Adam(net_d.parameters(), learning_rate_d, betas=(0.5,0.99))
        optimizer_g = T.optim.Adam(net_g.parameters(), learning_rate_g, betas=(0.5,0.99))
    if load_model:
        net_d.load_state_dict(T.load(path_work + 'net_d.pt'))
        net_g.load_state_dict(T.load(path_work + 'net_g.pt'))
    #
    iteration = 0
    for epoch in range(epochs):
        dataloader = DataLoader(Dataset_wgan(), batch_size=batch_size, shuffle=True, num_workers=num_workers)
        for data in dataloader:
            image_real = data.to(device)
            trainer_d(net_d, optimizer_d, net_g, image_real)
            iteration += 1
            if iteration % step_g == 0:
                trainer_g(net_g, optimizer_g, net_d, image_real)
    #
            if iteration % 1000 == 0:
                print('Epoch: ', epoch, ', Iteration: ', iteration)
                T.save(net_d.state_dict(), path_work + 'net_d.pt')
                T.save(net_g.state_dict(), path_work + 'net_g.pt')
                show(net_g)

In [ ]:
train(epochs=epochs, load_model=False)

In [ ]:
# train(epochs=epochs, load_model=True)

# Test

In [ ]:
net_g = Generator().to(device)
net_g.load_state_dict(T.load(path_work + 'net_g.pt'))
net_g.eval()
show(net_g, fix=False)